In [ ]:
#---IMPORTS---
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
plt.style.use('ggplot')


# **1. First look at the data**
Here we import the data, look at the column names, the data types. We may deal with quick fixes like null values, empty or useless columns.

In [ ]:
#---READING EXPORTING---
df=pd.read_csv('data.csv') # to read a csv file using pandas
df = pd.read_excel('data.xlsx', sheet_name=1)

df

In [ ]:
#---EXPLORING DATA NATURE---
df.columns # To print column names
df.dtypes # To print column dtypes

df=df.drop(columns=['uuid']) # to remove columns you don't need
df['date']=pd.to_datetime(df['date']) # when a date column does not have the correct dtype
df['total']=pd.to_numeric(df['total']) # when a numeric column does not have the correct dtype
df['Invoice'] = df['Invoice'].astype('str')

df.head() # print 1st 5 rows of df
df.tail() # print last 5 rows of df
df.describe() # print an statistical summary of the numeric columns data


In [ ]:

#---NULL VALUES---
sns.heatmap(df.isnull(), yticklabels=False, cbar=False)
df.isnull().sum() #to confirm the heatmap

df=df.dropna() #drops all rows that have at least one missing value
df=df.dropna(axis=1) #drops all columns that have at least one null value
df=df.dropna(how='all') #drops all rows or columns that are 100% null
df=df.dropna(thresh=3) #drops all rows with 3 or more null values
df=df.dropna(subset=['target_column']) #drops rows that have a null value in this column

df['column_with_nulls']=df['column_with_nulls'].fillna(0) #fills null values of that column with 0


In [ ]:
#---REMOVE DUPLICATES---
df=df.loc[~df.duplicated(subset=['column1', 'column2'])].copy()

In [ ]:
#---REPLACE ANYTHING---
df['col1'] = df['col1'].replace(0,None)


In [ ]:
# --- DATA MANIPULATION ----

df[['col1', 'col2']] # select columns

df[df['col'] > 5] # where condition or filtering
df[(df['col1'] > 5) & (df['col2'] < 10)]
df[(df['col1'] > 5) | (df['col2'] < 10)]

aggregated_df = df.groupby(by = 'Customer ID', as_index = False).agg( # grouping
    MonetaryValue = ('SalesLineTotal', "sum"),             # Total money customer has spent
    Frequency = ('Invoice', "nunique"),                    # Total purchases the customer has made
    LastInvoiceDate = ('InvoiceDate', "max"),              # Date of last purchase of the customer
    FirstInvoiceDate = ('InvoiceDate', "min"),             # Date of the first purchase the customer did
)
df.sort_values('col') # sorting data
df.sort_values('col', ascending=False)

pd.merge(df1, df2, on='key') # how to join tables
pd.merge(df1, df2, on='key', how='left')
pd.merge(df1, df2, on='key', how='right')
pd.merge(df1, df2, on='key', how='outer')

df.loc[len(df)] = df.append(...) # add row


# Row number per department based on score descending
df['row_number'] = df.sort_values(['department', 'score'], ascending=[True, False]) \
                     .groupby('department') \
                     .cumcount() + 1




# **2. Data Exploration**
Now we can do our 1st deep analysis of the data, we'll want to look carefully at each of the variables to find patterns that could be useful when preparing the data, removing outliers or for feature engineering.

In [ ]:
# prints a matrix of scatter plots and histograms looking at all the columns at the same time
sns.pairplot(data=df, vars=['meter_hours', 'category', 'manufacturer', 'region', 'price_usd', 'year'], hue='is_new')
plt.show()

# Looking at an individual scatter plot
ax=sns.scatterplot(data=df,x='meter_hours', y='price_usd', hue='is_new')
ax.set_title('price vs meter_hours')
plt.show()

# Looking at an individual distribution plot
ax=sns.displot(data=df,x='meter_hours', hue='is_new', binwidth=2000)
ax.fig.suptitle('meter_hours distribution')
plt.show()

# Looking at a box plot
ax=sns.boxplot(data=df, y='meter_hours', x='is_new')
ax.set_title('meter_hours box plot')
plt.show()

# Looking at a correlation heatmap
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

# Creating pivot plotss
pivot=pd.pivot_table(df, values='var_to_aggregate', index='what_you_are_groupingby', columns='what_you_want_to_segment_by', 
                     aggfunc=np.sum)
pivot.plot.line(figsize=(10,7), title='title', ylabel='Name')
plt.show()


# **3. Data Preparation and Feature Engineering**
After getting to know the data, we have a better idea of which outliers to remove and what new variables que can derive from our data.

In [ ]:
#---OUTLIERS---
df=df.loc[(df['meter_hours'] < df.meter_hours.quantile(0.992))].copy() #very particular way for removing some outliers of a potentially problematic column
df=df.loc[(df['price_usd'] < df.price_usd.quantile(0.95))].copy()

# or you can also use the Interquantile Range Method}
M_Q1 = df["MonetaryValue"].quantile(0.25)
M_Q3 = df["MonetaryValue"].quantile(0.75)
M_IQR = M_Q3 - M_Q1

monetary_outliers_df = df[(df["MonetaryValue"] > (M_Q3 + 1.5 * M_IQR)) | (df["MonetaryValue"] < (M_Q1 - 1.5 * M_IQR))].copy()


#--- WORKING WITH DATES ----
DF['Recency'] = (DF - aggregated_df['LastInvoiceDate']).dt.days # How recently has the customer made its latest purchase

In [ ]:

#---QUERY IN PYTHON---

# You can do complex queries this way if its easier for you to do it

from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

query='''
select *, 
(case when meter_hours <= 10000 then meter_hours*meter_hours else meter_hours end) as meter_hours_sq
from df
'''

df=pysqldf(query)
df['meter_hours_log'] = np.where(df['meter_hours'] > 1,np.log10(df['meter_hours']),np.nan)
df['year_sqrt'] = np.sqrt(df['year']).astype(int)
df

# **5. Model Creation**
From the nature of the problem you can identify which kind of algorithm you need (clasificator, regressor, clustering, anomalies, etc.) and with the data analysis you can know  which one of the many options of regressors to choose (in case it's a regressor): linear regression, XGBoostRegressor, lightGBMRegressor, DecisionTrees, RandomForest, etc.
<br/>
<br/>
We'll also experiment with different combinations for the hyperparameters of the models which will differ depending on the model.

In [ ]:
#---PREPARE TEST AND TRAIN DATA---
from sklearn.model_selection import train_test_split

x=df.drop(['price_usd'], axis=1)
y=df.price_usd

x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=4)


In [ ]:

#---USING XGBRegressor---
import xgboost as xgb

model=xgb.XGBRegressor(eta=0.2, max_depth=10, n_estimators=500)
model.fit(x_train.drop(columns=['meter_hours_sq', 'meter_hours_log']), y_train)


In [ ]:

# Make predictions on the test data
y_pred = model.predict(x_test.drop(columns=['meter_hours_sq', 'meter_hours_log']))


# **6. Model Evaluation**
There are different evaluation metrics that we can use depending on the nature of the algorithm the situation needs you to use. 
<br/>
<br/>
For regressors: R square, MSE, MAE, LMSQE.
<br/>
For classificators: Accuracy, precision, recall, ROC curve.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("EVALUATION FOR MODEL WITH ORIGINAL VARIABLES")
# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate the R-squared value
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)


Now steps 5 and 6 but for classification.

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier

# Load sample dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))  # e.g., Accuracy: 0.9561
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


# Plot feature importance
plt.figure(figsize=(10, 6))
xgb.plot_importance(model, max_num_features=10)
plt.title('Top 10 Feature Importances')
plt.tight_layout()
plt.show()

